# EDA: Market Analysis using Scala

Anik Chakraborty (waytoanik@outlook.com)
How to setup Jupyter Notebook to run Scala and Spark?

https://www.techentice.com/how-to-setup-jupyter-notebook-to-run-scala-and-spark/

DESCRIPTION

Background and Objective:

Your client, a Portuguese banking institution, ran a marketing campaign to convince potential customers to invest in a bank term deposit scheme. 
The marketing campaigns were based on phone calls. Often, the same customer was contacted more than once through phone, in order to assess if they would want to subscribe to the bank term deposit or not. You have to perform the marketing analysis of the data generated by this campaign.

In [1]:
spark.version

Intitializing Scala interpreter ...

Spark Web UI available at http://Anik:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1618673197431)
SparkSession available as 'spark'


res0: String = 3.1.1


In [2]:
sc.getConf.get("spark.driver.host")

res1: String = Anik


## 1. Load data and create a Spark data frame

In [3]:
val bankFile = spark.read.textFile("C:/Users/wayto/Desktop/Big Data/data.csv")
.map(_.stripPrefix("\"").stripSuffix("\""))
.map(_.replace("\"\"",""))

bankFile: org.apache.spark.sql.Dataset[String] = [value: string]


In [4]:
bankFile.show(5)

+--------------------+
|               value|
+--------------------+
|age;job;marital;e...|
|58;management;mar...|
|44;technician;sin...|
|33;entrepreneur;m...|
|47;blue-collar;ma...|
+--------------------+
only showing top 5 rows



In [5]:
val bankDf = spark.read.option("inferSchema","True")
    .option("header","True")
    .option("delimiter",";")
    .csv(bankFile)
    .toDF()

bankDf: org.apache.spark.sql.DataFrame = [age: int, job: string ... 15 more fields]


In [6]:
bankDf.printSchema

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [7]:
bankDf.show(5)

+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management|married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician| single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur|married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar|married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown| single|  unknown|     no|      1|     no|  no|unknown|  5|  may|     19

## 2. Give marketing success rate

In [8]:
bankDf.createOrReplaceTempView("bankView")

In [9]:
val campSubs = spark.sql("SELECT count(*) as cnt FROM bankView WHERE y='yes'").first.getAs[Long]("cnt")
val campAll = spark.sql("SELECT count(*) as cnt FROM bankView").first.getAs[Long]("cnt")

campSubs: Long = 5289
campAll: Long = 45211


In [10]:
val successPer = (campSubs.floatValue()/campAll.floatValue())*100
val failurePer = 100 - (campSubs.floatValue()/campAll.floatValue())*100

successPer: Float = 11.698481
failurePer: Float = 88.30152


In [11]:
println(s"Marketing success rate is: $successPer%")
println(s"Marketing failure rate is: $failurePer%") 

Marketing success rate is: 11.698481%
Marketing failure rate is: 88.30152%


## 3. Give the maximum, mean, and minimum age of the average targeted customer

In [12]:
val ageDetails = spark.sql("SELECT max(age) as maxAge, avg(age) as meanAge, min(age) as minAge FROM bankView")

ageDetails.show()

+------+-----------------+------+
|maxAge|          meanAge|minAge|
+------+-----------------+------+
|    95|40.93621021432837|    18|
+------+-----------------+------+



ageDetails: org.apache.spark.sql.DataFrame = [maxAge: int, meanAge: double ... 1 more field]


## 4. Check the quality of customers by checking average balance, median balance of customers

In [13]:
val balanceDetails = spark.sql("SELECT avg(balance) as averageBalance, percentile(balance, 0.5) as medianBalance FROM bankView")
balanceDetails.show()

+------------------+-------------+
|    averageBalance|medianBalance|
+------------------+-------------+
|1362.2720576850766|        448.0|
+------------------+-------------+



balanceDetails: org.apache.spark.sql.DataFrame = [averageBalance: double, medianBalance: double]


## 5. Check if age matters in marketing subscription for deposit

In [14]:
val ageGrouped = spark.sql("SELECT age, y, count(*) as cnt FROM bankView GROUP BY age,y ORDER BY age,y DESC")
ageGrouped.show()

+---+---+---+
|age|  y|cnt|
+---+---+---+
| 18|yes|  7|
| 18| no|  5|
| 19|yes| 11|
| 19| no| 24|
| 20|yes| 15|
| 20| no| 35|
| 21|yes| 22|
| 21| no| 57|
| 22|yes| 40|
| 22| no| 89|
| 23|yes| 44|
| 23| no|158|
| 24|yes| 68|
| 24| no|234|
| 25|yes|113|
| 25| no|414|
| 26|yes|134|
| 26| no|671|
| 27|yes|141|
| 27| no|768|
+---+---+---+
only showing top 20 rows



ageGrouped: org.apache.spark.sql.DataFrame = [age: int, y: string ... 1 more field]


Above data is not giving us much clear view, instead let's create some age groups of 18-30, 31-60, 61-95 (Min value of Age in the dataset is 18 and max value is 95)


In [15]:
val ageGp = spark.sql(""" SELECT CASE 
        WHEN Age>= 18 AND Age <= 30 THEN '18-30' 
        WHEN Age>= 31 AND Age <= 60 THEN '31-60' 
        WHEN Age>= 61 AND Age <= 95 THEN '61-95' 
        ELSE 'Other' END as ageGps, y as Subscribed, count(*) as counts FROM bankView GROUP BY ageGps, y ORDER BY ageGps, y DESC """)

ageGp.show()

+------+----------+------+
|ageGps|Subscribed|counts|
+------+----------+------+
| 18-30|       yes|  1145|
| 18-30|        no|  5885|
| 31-60|       yes|  3642|
| 31-60|        no| 33351|
| 61-95|       yes|   502|
| 61-95|        no|   686|
+------+----------+------+



ageGp: org.apache.spark.sql.DataFrame = [ageGps: string, Subscribed: string ... 1 more field]


It can be observed that this campaign has targeted age group 31-60 the most and this age group has the highest subscribers though campaign success rate is higher for age group 18-30.

## 6. Check if marital status mattered for a subscription to deposit

In [16]:
val maritalGrouped = spark.sql("SELECT marital, y as Subscribed, count(*) as counts FROM bankView GROUP BY marital,y ORDER BY marital,y DESC")
maritalGrouped.show()

+--------+----------+------+
| marital|Subscribed|counts|
+--------+----------+------+
|divorced|       yes|   622|
|divorced|        no|  4585|
| married|       yes|  2755|
| married|        no| 24459|
|  single|       yes|  1912|
|  single|        no| 10878|
+--------+----------+------+



maritalGrouped: org.apache.spark.sql.DataFrame = [marital: string, Subscribed: string ... 1 more field]


In [17]:
//campaign success rate for divorced

val divorcedY = maritalGrouped.filter($"marital" === "divorced").filter($"Subscribed" === "yes").first.getAs[Long]("counts").floatValue()
val divorcedN = maritalGrouped.filter($"marital" === "divorced").filter($"Subscribed" === "no").first.getAs[Long]("counts").floatValue()
val divorcedPer = (divorcedY/(divorcedY+divorcedN))*100

divorcedY: Float = 622.0
divorcedN: Float = 4585.0
divorcedPer: Float = 11.945457


In [18]:
//campaign success rate for married

val marriedY = maritalGrouped.filter($"marital" === "married").filter($"Subscribed" === "yes").first.getAs[Long]("counts").floatValue()
val marriedN = maritalGrouped.filter($"marital" === "married").filter($"Subscribed" === "no").first.getAs[Long]("counts").floatValue()
val marriedPer = (marriedY/(marriedY+marriedN))*100

marriedY: Float = 2755.0
marriedN: Float = 24459.0
marriedPer: Float = 10.123466


In [19]:
//campaign success rate for single

val singleY = maritalGrouped.filter($"marital" === "single").filter($"Subscribed" === "yes").first.getAs[Long]("counts").floatValue()
val singleN = maritalGrouped.filter($"marital" === "single").filter($"Subscribed" === "no").first.getAs[Long]("counts").floatValue()
val singlePer = (singleY/(singleY+singleN))*100

singleY: Float = 1912.0
singleN: Float = 10878.0
singlePer: Float = 14.949179


In [20]:
println(s"Campaign success rate for: \n Divorced is: $divorcedPer% \n Married is: $marriedPer%  \n Single is: $singlePer% ")

Campaign success rate for: 
 Divorced is: 11.945457% 
 Married is: 10.123466%  
 Single is: 14.949179% 


### Observations 1:
1. This campaign targeted Married customers the most, then followed by Single and Divorced.
2. Campaign success rate for Single customers is the highest. So more single customers should have been considered for this campaign.
3. There is no significant different between campaign success rate among Divorced and Married customers.

## 7. Check if age and marital status together mattered for a subscription to deposit scheme

In [21]:
val ageMarital = spark.sql(""" SELECT CASE 
        WHEN Age>= 18 AND Age <= 30 THEN '18-30' 
        WHEN Age>= 31 AND Age <= 60 THEN '31-60' 
        WHEN Age>= 61 AND Age <= 95 THEN '61-95' 
        ELSE 'Other' END as ageGroups, marital, y as Subscribed, count(*) as counts FROM bankView GROUP BY ageGroups, marital, y ORDER BY ageGroups, marital, y DESC """)

ageMarital.show()

+---------+--------+----------+------+
|ageGroups| marital|Subscribed|counts|
+---------+--------+----------+------+
|    18-30|divorced|       yes|    18|
|    18-30|divorced|        no|   152|
|    18-30| married|       yes|   182|
|    18-30| married|        no|  1879|
|    18-30|  single|       yes|   945|
|    18-30|  single|        no|  3854|
|    31-60|divorced|       yes|   507|
|    31-60|divorced|        no|  4331|
|    31-60| married|       yes|  2177|
|    31-60| married|        no| 22021|
|    31-60|  single|       yes|   958|
|    31-60|  single|        no|  6999|
|    61-95|divorced|       yes|    97|
|    61-95|divorced|        no|   102|
|    61-95| married|       yes|   396|
|    61-95| married|        no|   559|
|    61-95|  single|       yes|     9|
|    61-95|  single|        no|    25|
+---------+--------+----------+------+



ageMarital: org.apache.spark.sql.DataFrame = [ageGroups: string, marital: string ... 2 more fields]


### Observations 2:

1. For age group 18-30, Single customers are the most subscribed customers as well as the campaign success rate is the highest for them.
2. For age group 31-60, though most of the subscribed customers are Married, but success rate is a bit higher for Single customers.
3. For age group 61-95, number of targeted customers are lesser that other groups. In this age group Married customers have subscribed the most but campaign success rate is highest for Divorced customers.

## 8. Do feature engineering for the bank and find the right age effect on the campaign

Creating 7 Age Groups to do a detailed analysis to identify the age effect on this campaign:

In [22]:
val ageBin = spark.sql(""" SELECT CASE 
        WHEN Age>= 18 AND Age <= 30 THEN '18-30' 
        WHEN Age>= 31 AND Age <= 40 THEN '31-40' 
        WHEN Age>= 41 AND Age <= 50 THEN '41-50' 
        WHEN Age>= 51 AND Age <= 60 THEN '51-60' 
        WHEN Age>= 61 AND Age <= 70 THEN '61-70' 
        WHEN Age>= 71 AND Age <= 80 THEN '71-80' 
        WHEN Age>= 81 AND Age <= 95 THEN '81-95' 
        ELSE 'Other' END as ageGroups, y as Subscribed, count(*) as counts FROM bankView GROUP BY ageGroups,y 
        ORDER BY ageGroups,y DESC """)

ageBin.show()

+---------+----------+------+
|ageGroups|Subscribed|counts|
+---------+----------+------+
|    18-30|       yes|  1145|
|    18-30|        no|  5885|
|    31-40|       yes|  1812|
|    31-40|        no| 15875|
|    41-50|       yes|  1019|
|    41-50|        no| 10220|
|    51-60|       yes|   811|
|    51-60|        no|  7256|
|    61-70|       yes|   284|
|    61-70|        no|   417|
|    71-80|       yes|   175|
|    71-80|        no|   213|
|    81-95|       yes|    43|
|    81-95|        no|    56|
+---------+----------+------+



ageBin: org.apache.spark.sql.DataFrame = [ageGroups: string, Subscribed: string ... 1 more field]


#### Let's analyze the success rate of 18-30, 31-40, 41-50, 51-60 these 4 age groups:

In [23]:
//campaign success rate for age group 18-30

val age18Y = ageBin.filter($"ageGroups" === "18-30").filter($"Subscribed" === "yes").first.getAs[Long]("counts").floatValue()
val age18N = ageBin.filter($"ageGroups" === "18-30").filter($"Subscribed" === "no").first.getAs[Long]("counts").floatValue()
val success18Per = (age18Y/(age18Y+age18N))*100

age18Y: Float = 1145.0
age18N: Float = 5885.0
success18Per: Float = 16.28734


In [24]:
//campaign success rate for age group 31-40

val age31Y = ageBin.filter($"ageGroups" === "31-40").filter($"Subscribed" === "yes").first.getAs[Long]("counts").floatValue()
val age31N = ageBin.filter($"ageGroups" === "31-40").filter($"Subscribed" === "no").first.getAs[Long]("counts").floatValue()
val success31Per = (age31Y/(age31Y+age31N))*100

age31Y: Float = 1812.0
age31N: Float = 15875.0
success31Per: Float = 10.244813


In [25]:
//campaign success rate for age group 41-50

val age41Y = ageBin.filter($"ageGroups" === "41-50").filter($"Subscribed" === "yes").first.getAs[Long]("counts").floatValue()
val age41N = ageBin.filter($"ageGroups" === "41-50").filter($"Subscribed" === "no").first.getAs[Long]("counts").floatValue()
val success41Per = (age41Y/(age41Y+age41N))*100

age41Y: Float = 1019.0
age41N: Float = 10220.0
success41Per: Float = 9.066643


In [26]:
//campaign success rate for age group 51-60

val age51Y = ageBin.filter($"ageGroups" === "51-60").filter($"Subscribed" === "yes").first.getAs[Long]("counts").floatValue()
val age51N = ageBin.filter($"ageGroups" === "51-60").filter($"Subscribed" === "no").first.getAs[Long]("counts").floatValue()
val success51Per = (age51Y/(age51Y+age51N))*100

age51Y: Float = 811.0
age51N: Float = 7256.0
success51Per: Float = 10.053304


In [27]:
println(s"Campaign success rate for age group 18-30 is: $success18Per%")
println(s"Campaign success rate for age group 31-40 is: $success31Per%")
println(s"Campaign success rate for age group 41-50 is: $success41Per%")
println(s"Campaign success rate for age group 51-60 is: $success51Per%")

Campaign success rate for age group 18-30 is: 16.28734%
Campaign success rate for age group 31-40 is: 10.244813%
Campaign success rate for age group 41-50 is: 9.066643%
Campaign success rate for age group 51-60 is: 10.053304%


### Observations 3:

1. This campaign targeted age group of 31-40 the most and then followed by 41-50, 51-60, 18-30 and other age groups.
2. Most of the subscribed customers belong to the age group of 31-40, followed by 18-30, 41-50, 51-60 age groups.
3. This Campaign has an overall success rate 11.698481%, where the success rate for the age group 18-30 is 16.28734%. Age group 51-60 and 31-40 have almost the same success rate. It can be concluded that more focus should have been given to 18-30 age group. This campaign underestimated the age group 18-30 as a potential customer to invest in a bank term deposit.